In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import pickle as pkl
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression 
from sklearn.model_selection import train_test_split
import joblib

%run ../dnam-clocks/mFSS-clock/scripts/mfss_model_train.py
%matplotlib inline